In [2]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [3]:
df = pd.read_csv('./compras.csv')

df.head()

,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,MAYCON SANTOS,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

new_df = df.drop(['cliente'])

df.stack().unique()

KeyError: "['cliente'] not found in axis"

In [8]:
new_df = df.replace('vinho tinto', 'vinho')
new_df = new_df.replace('vinho branco', 'vinho')

new_df.stack().unique()

array(['JÚLIO DEMARQUE', 'limão', 'peito de peru', 'manga', 'cerveja',
       'picanha', 'carvão', 'sal', 'linguiça', 'mel', 'geleia', 'cereal',
       'MAYCON SOARES', 'salsicha', 'catchup', 'cebola', 'WEMERSON NETO',
       'queijo', 'frutas', 'chá', 'alface', 'granola', 'feijão',
       'farinha de trigo', 'suco de laranja', 'JOÃO DEMARQUE', 'pimenta',
       'sardinha enlatada', 'barra de cereal', 'molho de tomate',
       'gelatina', 'vinho', 'pão', 'tempero pronto', 'presunto', 'arroz',
       'MAYCON SANTOS', 'detergente', 'filé de frango', 'papel higiênico',
       'salada', 'cenoura', 'PEDRO SOARES', 'maionese', 'OTÁVIO NETO',
       'molho branco', 'whisky', 'pudim', 'suco', 'azeitona', 'frango',
       'VICTOR NETO', 'GABRIEL NETO', 'mostarda', 'carne moída',
       'MAYCON OLIVEIRA', 'nhoque', 'água de coco', 'laranja',
       'MAYCON SILVA', 'cerveja artesanal', 'massa de pizza', 'biscoito',
       'MAYCON RODRIGUES', 'morango', 'bisnaguinha', 'kiwi',
       'GABRIEL SANTO

In [9]:
# itens com mais compras
itens_mais_comprados = new_df.stack().value_counts()
# 20 itens mais comprados
print(itens_mais_comprados)

queijo             1241
vinho              1178
presunto           1129
pão                1056
feijão             1040
                   ... 
PEDRO LIMA            7
VANESSA NÁPOLES       7
DANIEL MASSI          6
LUIZ MASSI            4
VICTOR CRUZ           3
Name: count, Length: 338, dtype: int64


In [10]:
transactions = []
for i in range(0,new_df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

In [11]:
regras = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)

In [12]:
resultados = list(regras)
resultados



[RelationRecord(items=frozenset({'azeite', 'alface'}), support=0.0104, ordered_statistics=[OrderedStatistic(items_base=frozenset({'azeite'}), items_add=frozenset({'alface'}), confidence=0.20392156862745098, lift=3.3594986594308236)]),
 RelationRecord(items=frozenset({'alface', 'cenoura'}), support=0.0129, ordered_statistics=[OrderedStatistic(items_base=frozenset({'alface'}), items_add=frozenset({'cenoura'}), confidence=0.21252059308072488, lift=3.885202798550729), OrderedStatistic(items_base=frozenset({'cenoura'}), items_add=frozenset({'alface'}), confidence=0.23583180987202926, lift=3.885202798550729)]),
 RelationRecord(items=frozenset({'salada', 'alface'}), support=0.0254, ordered_statistics=[OrderedStatistic(items_base=frozenset({'alface'}), items_add=frozenset({'salada'}), confidence=0.4184514003294893, lift=5.096850186717287), OrderedStatistic(items_base=frozenset({'salada'}), items_add=frozenset({'alface'}), confidence=0.3093788063337393, lift=5.096850186717287)]),
 RelationRecor

In [14]:
df_compras_lista = new_df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

rules.size

# imprimindo as regras mais importantes ordenadas por lift
rules = rules.sort_values(by='lift', ascending=False)

print(rules)

                            antecedents                        consequents  \
810   (mostarda, batata frita, cerveja)              (hambúrguer, catchup)   
823               (hambúrguer, catchup)  (mostarda, batata frita, cerveja)   
816    (catchup, batata frita, cerveja)             (mostarda, hambúrguer)   
817              (mostarda, hambúrguer)   (catchup, batata frita, cerveja)   
1233                   (azeite, salada)         (vinagre, tomate, cenoura)   
...                                 ...                                ...   
986                            (açúcar)                            (leite)   
688                      (batata frita)                          (cerveja)   
689                           (cerveja)                     (batata frita)   
101                            (geleia)                 (queijo, presunto)   
100                  (queijo, presunto)                           (geleia)   

      antecedent support  consequent support  support  confiden